In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install emoji
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large", normalization=True)

In [ ]:
import pandas as pd
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df = df.rename({'target':'label'}, axis = 1)
df = df[['text', 'label']]
df['text'] = df['text'].apply(lambda x: preprocess(x))
df = df.sample(frac = 1)
df.to_csv('train_processed.csv')

df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_test['text'] = df_test['text'].apply(lambda x: preprocess(x))
df_test = df_test[['text']]
df_test.to_csv('test_processed.csv')

In [ ]:
from datasets import load_dataset
# train_dataset = load_dataset('csv', data_files='train_processed.csv', split = 'train[:80%]')
# val_dataset = load_dataset('csv', data_files='train_processed.csv', split = 'train[80%:]')
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", max_length=100)
# tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
# tokenized_datasets_val = val_dataset.map(tokenize_function, batched=True)
train_dataset = load_dataset('csv', data_files='train_processed.csv', split='train[:90%]')
val_dataset = load_dataset('csv', data_files='train_processed.csv', split='train[90%:]')
test_dataset = load_dataset('csv', data_files='test_processed.csv', split = 'train')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=100)
tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_val = val_dataset.map(tokenize_function, batched=True)
tokenized_datasets_test = test_dataset.map(tokenize_function, batched=True)



In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=2)


In [ ]:
from transformers import TrainingArguments

#training_args = TrainingArguments("test_trainer")

In [ ]:
training_args = TrainingArguments(
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
gradient_accumulation_steps=8,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
load_best_model_at_end=False,
local_rank=-1,
max_grad_norm=1.0,
max_steps=-1,
no_cuda=False,
num_train_epochs=10.0,
output_dir="test_trainer2",
overwrite_output_dir=False,
past_index=-1,
per_device_eval_batch_size=32,
per_device_train_batch_size=32,
prediction_loss_only=False,
remove_unused_columns=True,
resume_from_checkpoint=None,
save_on_each_node=False,
save_steps = 100,
save_total_limit=None,
seed=42,
sharded_ddp=[],
skip_memory_metrics=True,
tpu_metrics_debug=False,
tpu_num_cores=None,
use_legacy_prediction_loop=False,
warmup_ratio=0.0,
warmup_steps=0,
weight_decay=0.0,
xpu_backend=None,
evaluation_strategy = "epoch",
)

In [ ]:
from transformers import Trainer
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets_train, eval_dataset=tokenized_datasets_val, compute_metrics=compute_metrics, 
)

In [ ]:
trainer.train()

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./test_trainer2/checkpoint-100", num_labels=2)


In [ ]:
cd test_trainer2

In [ ]:
ls

In [ ]:
trainer.evaluate()

In [ ]:
predictions = trainer.predict(tokenized_datasets_test)

In [ ]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)


In [ ]:
from collections import Counter
Counter(preds)

In [ ]:
import pandas as pd 
df_sub = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
df_sub['target'] = preds

In [ ]:
df_sub.to_csv('/kaggle/working/bertweet-large-submission.csv', index = False)

In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/bertweet_submission_epoch2.csv')

In [ ]:
cd working

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.') 